In [ ]:
# default_exp core
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# core

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from fastai.vision.all import *
from fastai.medical.imaging import *
from torchvision.utils import save_image

### Instance Sort

In [ ]:
#export
def instance_sort(folder:(Path, L)):
    "sort files by instance number"
    if isinstance(folder, Path): folder = get_dicom_files(folder)
    if isinstance(folder, L): folder = folder
    sorted_files = []
    for file in folder:
        instance = dcmread(file)[0x20, 0x13].value
        sorted_files.append([instance, file])
    return L(sorted(sorted_files))

In [ ]:
show_doc(instance_sort)

<h4 id="instance_sort" class="doc_header"><code>instance_sort</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>instance_sort</code>(**`folder`**:`L'>)`)

sort files by instance number

In [ ]:
#export
def instance_dcmread(folder:(L)):
    "instance dcmread"
    file = [dcmread(o[1]) for o in folder]
    return file

In [ ]:
show_doc(instance_dcmread)

<h4 id="instance_dcmread" class="doc_header"><code>instance_dcmread</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>instance_dcmread</code>(**`folder`**:`L`)

instance dcmread

In [ ]:
#export
def instance_show(folder: (L), nrows=1):
    "show sorted files"
    f_list = []
    for file in instance_sort(folder):
        f = TensorDicom(dcmread(file[1]).pixel_array)
        f_list.append(f)
    return show_images(f_list, nrows=nrows)

In [ ]:
show_doc(instance_show)

<h4 id="instance_show" class="doc_header"><code>instance_show</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>instance_show</code>(**`folder`**:`L`, **`nrows`**=*`1`*)

show sorted files

### View images

In [ ]:
#export
def get_dicom_image(df, key, nrows=3, source=None):
    imgs=[]
    title=[]
    for i in df.index:
        file_path = f"{source}/{df.iloc[i]['PatientID']}/{df.iloc[i]['InstanceNumber']}.dcm"
        dcc = dcmread(file_path).pixel_array
        imgs.append(dcc)
        pct = df.iloc[i][key]
        title.append(pct)
    return show_images(imgs, titles=title, nrows=nrows)

In [ ]:
show_doc(get_dicom_image)

<h4 id="get_dicom_image" class="doc_header"><code>get_dicom_image</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>get_dicom_image</code>(**`df`**, **`key`**, **`nrows`**=*`3`*, **`source`**=*`None`*)



### Mask & Save

In [ ]:
#export
def mask_and_save(df, source=None, show=None, window=dicom_windows.lungs, sigma:float=0.1, thresh:float=0.9, save=False, save_path=None):
    image_list = []
    for i in df.index:
        file_path = f"{source}/{df.iloc[i]['PatientID']}/{df.iloc[i]['InstanceNumber']}.dcm"
        file_name = df.iloc[i]['InstanceNumber']
        dcm = dcmread(file_path)
        wind = dcm.windowed(*window)
        mask = dcm.mask_from_blur(window, sigma=sigma, thresh=thresh, remove_max=False)
        bbs = mask2bbox(mask)
        lo,hi = bbs
        imh = wind[lo[0]:hi[0],lo[1]:hi[1]]
        if save is not False:
            save_image(imh, f'{save_path}/{file_name}.png')
        else:
            pass
        image_list.append(imh)
    if show is not None:
        show_images(image_list[:10], nrows=1)

In [ ]:
show_doc(mask_and_save)

<h4 id="mask_and_save" class="doc_header"><code>mask_and_save</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>mask_and_save</code>(**`df`**, **`source`**=*`None`*, **`show`**=*`None`*, **`window`**=*`(1500, -600)`*, **`sigma`**:`float`=*`0.1`*, **`thresh`**:`float`=*`0.9`*, **`save`**=*`False`*, **`save_path`**=*`None`*)



### Dicom metadata dict

Change to allow selection of dicom window width and level

In [ ]:
#export
@patch
def updated_dict(self:DcmDataset, windows=[dicom_windows.lungs]):
    pxdata = (0x7fe0,0x0010)
    vals = [self[o] for o in self.keys() if o != pxdata]
    its = [(v.keyword, v.value) for v in vals]
    res = dict(its)
    
    stats = 'min', 'max', 'mean', 'std'
    pxs = self.pixel_array
    for f in stats: res['img_'+f] = getattr(pxs, f)()
    res['img_pct_window'] = self.pct_in_window(*windows)
    res['file_path'] = f'{self.PatientID}/{self.InstanceNumber}.dcm'
    return res

In [ ]:
#export
def _dcm2dict2(fn, windows, **kwargs): return fn.dcmread().updated_dict(windows, **kwargs)

In [ ]:
#export
@delegates(parallel)
def _from_dicoms2(cls, fns, n_workers=0, **kwargs):
    return pd.DataFrame(parallel(_dcm2dict2, fns, n_workers=n_workers, **kwargs))
pd.DataFrame.from_dicoms2 = classmethod(_from_dicoms2)

### dicom convert 3channel

In [ ]:
#export
def dicom_convert_3channel(fn:(Path,str), save_dir:(str), win1=dicom_windows.lungs, \
                           win2=dicom_windows.liver, win3=dicom_windows.brain):
    "Split a dicom image into 3 windows with each window per channel and saved as jpg"
    data = dcmread(fn)
    file_name = str(fn); name = file_name.split('\\')[-1].split('.')[0]
        
    chan_one = np.expand_dims(data.windowed(*win1), axis=2)
    chan_two = np.expand_dims(data.windowed(*win2), axis=2)
    chan_three = np.expand_dims(data.windowed(*(win3)), axis=2)
    image = np.concatenate([chan_one, chan_two, chan_three], axis=2)
    ten_image = TensorImage(image).permute(2,0,1)
    save_image(ten_image, f'{save_dir}/{name}.jpg')

In [ ]:
show_doc(dicom_convert_3channel)

<h4 id="dicom_convert_3channel" class="doc_header"><code>dicom_convert_3channel</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>dicom_convert_3channel</code>(**`fn`**:`Path'>, <class 'str'>)`, **`save_dir`**:`str`, **`win1`**=*`(1500, -600)`*, **`win2`**=*`(150, 30)`*, **`win3`**=*`(80, 40)`*)

Split a dicom image into 3 windows with each window per channel and saved as jpg

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
